# Práctica 3

**Nombre:** Gutierrez Ramirez Felipe de Jesus

**e-mail:** felipe.gutierrez5025@alumnos.udg.mx

## MODULES

In [1]:
import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

from scipy.spatial import distance

## CLASSES

In [3]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## FUNCTIONS

In [4]:
###############################################################################################
# Turning angle
# This function calculates the turning angle between three consecutive positions
###############################################################################################
def turning_angle(pos_a, pos_b, pos_c):
    """
    Arguments:
        pos_a: First position coordinates
        pos_b: Second position coordinates
        pos_c: Third position coordinates
    Returns:
        theta: Turning angle
    """
    vec_ab = np.array(pos_b) - np.array(pos_a)
    norm_ab = np.linalg.norm(vec_ab)

    vec_bc = np.array(pos_c) - np.array(pos_b)
    norm_bc = np.linalg.norm(vec_bc)

    dot_p = np.dot(vec_ab, vec_bc)

    # Nota: Evitar division por cero con np.finfo(float).eps
    cos_theta = dot_p / (norm_ab * norm_bc + np.finfo(float).eps)

    # Angle orientation
    cross_p = np.cross(vec_ab, vec_bc)
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1

    theta = np.arccos(np.around(cos_theta,4)) * orient
    return theta



In [5]:
######################################################################
# Brownian Motion Trajectory
######################################################################

def bm_2d(n_steps=1000, speed=5, s_pos=[0,0]):
  """
  Arguments:
    n_steps:
    speed:
    s_pos:
  Returns:
    BM_2d_df:
  """

  # Init velocity vector
  velocity = Vec2d(speed, 0)

  BM_2d_df = pd.DataFrame(columns = ['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])

  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i]+velocity.x, 'y_pos': BM_2d_df.y_pos[i]+velocity.y}])

    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  return BM_2d_df

In [45]:
######################################################################
# Correlated Random Walk Trajectory
######################################################################

def crw(CRW_exponent=0.4, n_steps=1000, speed=5, s_pos=[0,0]):
  """
  Arguments:
    CRW_exponent:
    n_steps:
    speed:
    s_pos:
  Returns:
    CRW_df:
  """
  velocity = Vec2d(speed,0)
  pos = Vec2d(s_pos)
  cauchy_samples = wrapcauchy.rvs(CRW_exponent, size=n_steps, loc=0)

  cauchy = pd.DataFrame(columns=['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])
  cauchy = pd.concat([cauchy,temp_df], ignore_index=True)

  for i in range(n_steps-1):
    # Elegir el angulo de giro
    velocity = velocity.rotated(cauchy_samples[i])
    pos += velocity
    temp_df = pd.DataFrame([{'x_pos': pos.x, 'y_pos': pos.y}])
    cauchy = pd.concat([cauchy,temp_df], ignore_index=True)

  return cauchy

## Actividad 1: Path Length - (BM1 vs BM2 vs CRW) (4 pts)

* Implementar función que genere **Brownian Motions** (BM) utilizando **pandas**.
* Implementar función que genere **Correlated Random Walks** (CRW) utilizando pandas.
* Implementar una función alternativa a las ya disponibles en los distintos modulos de python que calcule los valores de la curva de **path length** de una trayectoria.
* Guardar los valores de la métrica en un Data Frame de **pandas**.
* Visualizar con **plotly**.

In [6]:
# Load existing trajectories to test your implementation
# BM speed = 3
BM_2d_df_3 = pd.read_csv('files/trajectories/brownian_3.csv')

# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('files/trajectories/brownian_6.csv')

# Load existing trajectories to test your implementation
CRW_2d_df_9 = pd.read_csv('files/trajectories/crw_6_9.csv')

BM_3 = bm_2d(1000, 3, [0,0])
BM_6 = bm_2d(1000, 6, [0,0])
CR_9 = crw(0.9, 1000, 6, [0,0])

In [5]:
# Define your function to compute path length for given trajectory

## start - Add your code here
def path_length(traj):
    lengths = pd.DataFrame(columns=['distance'])
    temp_df = pd.DataFrame([{'distance': 0}])
    lengths =pd.concat([lengths,temp_df], ignore_index=True)
    for i in range(1, len(traj)):
        x0, y0 = traj.iloc[i - 1]['x_pos'], traj.iloc[i - 1]['y_pos']
        x1, y1 = traj.iloc[i]['x_pos'], traj.iloc[i]['y_pos']
        length = lengths.iloc[-1]['distance'] + np.sqrt((x1 - x0)**2 + (y1 - y0)**2)
        temp_df = pd.DataFrame([{'distance': length}])
        lengths =pd.concat([lengths,temp_df], ignore_index=True)
    return lengths

## end - Add your code here

In [8]:
# Get Path length calling the function
PL_BM_3 = path_length(BM_2d_df_3) 

# Get Path length calling the function
PL_BM_6 = path_length(BM_2d_df_6)

# Get Path length calling the function
PL_CRW_6 = path_length(CRW_2d_df_9)

PL_BM_3_G = path_length(BM_3)
PL_BM_6_G = path_length(BM_6)
PL_CRW_6_G = path_length(CR_9)

In [ ]:
# Plotting
# Init figure
fig_path_length = go.Figure()
fig_path_length_G = go.Figure()

# First trace BM1
## start - Add your code here
fig_path_length.add_trace(go.Scatter(x = np.arange(len(PL_BM_3['distance']))+1,
                                     y=PL_BM_3['distance'], 
                                     mode='lines', 
                                     name='PL_BM_3',
                                     showlegend = True))

fig_path_length_G.add_trace(go.Scatter(x = np.arange(len(PL_BM_3_G['distance']))+1,
                                     y=PL_BM_3_G['distance'], 
                                     mode='lines', 
                                     name='PL_BM_3_G',
                                     showlegend = True))
## end - Add your code here

# Second trace BM2
## start - Add your code here
fig_path_length.add_trace(go.Scatter(x = np.arange(len(PL_BM_6['distance']))+1,
                                     y=PL_BM_6['distance'], 
                                     mode='lines', 
                                     line = dict(width = 6),
                                     name='PL_BM_6',
                                     showlegend = True))

fig_path_length_G.add_trace(go.Scatter(x = np.arange(len(PL_BM_6_G['distance']))+1,
                                     y=PL_BM_6_G['distance'], 
                                     mode='lines', 
                                     line = dict(width = 6),
                                     name='PL_BM_6_G',
                                     showlegend = True))
## end - Add your code here

# Third trace CRW
## start - Add your code here
fig_path_length.add_trace(go.Scatter(x = np.arange(len(PL_CRW_6['distance']))+1,
                                     y=PL_CRW_6['distance'], 
                                     mode='lines', 
                                     name='PL_CRW_6',
                                     showlegend = True))

fig_path_length_G.add_trace(go.Scatter(x = np.arange(len(PL_CRW_6_G['distance']))+1,
                                     y=PL_CRW_6_G['distance'], 
                                     mode='lines', 
                                     name='PL_CRW_6_G',
                                     showlegend = True))
## end - Add your code here

fig_path_length.show()
fig_path_length_G.show()

## Actividad 2: Mean Squared Displacement - (Brownian vs CRW) (4 pts)

* Generar una trayectoria tipo **BM** y una **CRW**.
* Implementar una función que calcule los valores de la curva de **mean squared displacement** de una trayectoria.
* Guardar metricas en Pandas Data Frame.
* Visualizar con **plotly**.

In [50]:
# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('files/trajectories/brownian_6.csv')
# Load existing trajectories to test your implementation
# CRW speed = 6, c = 0.9
CRW_2d_df_9 = pd.read_csv('files/trajectories/crw_6_9.csv')

In [ ]:
# Show trajectories
# Init figure
fig_3d = go.Figure()

# Plot trajectory in 3-D space
fig_3d.add_trace(
    go.Scatter3d(x = BM_2d_df_6.x_pos,
                 y = BM_2d_df_6.y_pos,
                 z = BM_2d_df_6.index,
                 marker = dict(size=2),
                 line = dict(color='blue', width=2),
                 mode = 'lines',
                 name = 'BM 2d',
                 showlegend = True))


fig_3d.add_trace(
    go.Scatter3d(x = CRW_2d_df_9.x_pos,
                 y = CRW_2d_df_9.y_pos,
                 z = CRW_2d_df_9.index,
                 marker = dict(size=2),
                 line = dict(color='red', width=2),
                 mode = 'lines',
                 name = 'CRW 2d',
                 showlegend = True))

fig_3d.show()

In [80]:
# Define your function to compute Mean Squared Displacement for given trajectory

## start - Add your code here
def msd(traj):
    N = len(traj)
    msd_list = []
    for i in range(1, N):
        squared_displacements = (traj.iloc[i:] - traj.iloc[:-i].values) ** 2
        msd = squared_displacements.sum(axis=1).mean()
        msd_list.append(msd)
    msd = pd.DataFrame({'MSD': msd_list})
    return msd

## end - Add your code here

In [83]:
# Get Mean Squared Displacement calling the function
MSD_BM = msd(BM_2d_df_6)
# Get Mean Squared Displacement calling the function
MSD_CRW = msd(CRW_2d_df_9)

In [85]:
BM_2d_df_6_G = bm_2d(1000, 6, [0,0])
CRW_2d_df_9_G = crw(0.9, 1000, 6, [0,0])

MSD_BM_G = msd(BM_2d_df_6_G)
MSD_CRW_G = msd(CRW_2d_df_9_G)

In [86]:
# Init figure
fig_path_length = go.Figure()
fig_path_length_G = go.Figure()

# first trace MSD_BM
## start - Add your code here
fig_path_length.add_trace(go.Scatter(
    x = MSD_BM.index,
    y = MSD_BM.MSD,
    name = 'MSD BM 6',
    showlegend = True
))

fig_path_length_G.add_trace(go.Scatter(
    x = MSD_BM_G.index,
    y = MSD_BM_G.MSD,
    name = 'MSD BM 6 Generated',
    showlegend = True
))
## end - Add your code here


# Second trace MSD_CRW
## start - Add your code here
fig_path_length.add_trace(go.Scatter(
    x = MSD_CRW.index,
    y = MSD_CRW.MSD,
    name = 'MSD CRW 6',
    showlegend = True
))

fig_path_length_G.add_trace(go.Scatter(
    x = MSD_CRW_G.index,
    y = MSD_CRW_G.MSD,
    name = 'MSD CRW 6 Generated',
    showlegend = True
))
## end - Add your code here


fig_path_length.show()
fig_path_length_G.show()

## Actividad 3: Turning-angle Distribution - (Dist. origen vs Dist. observada) (6 pts)

* Generar **CRWs** con dos exponentes diferentes
* Guardar trayectorias en **Pandas** Data Frame
* Implementar una función que calcule los valores de **turning angle** de una trayectoria.
* Comparar en gráfica distribución origen vs distribución observada (Histograma)
* Visualizar con **plotly**

In [46]:
# Load existing trajectories to test your implementation
# CRW speed = 6,
# wrapcauchy [c = 0.6]
#CRW_2d_df_6 = pd.read_csv('files/trajectories/crw_6_6.csv')

# Load existing trajectories to test your implementation
# CRW speed = 6,
# wrapcauchy [c = 0.9]
#CRW_2d_df_9 = pd.read_csv('files/trajectories/crw_6_9.csv')

CRW_2d_df_6 = crw(CRW_exponent=0.6, n_steps=1000, speed=6, s_pos=[0,0])
CRW_2d_df_9 = crw(CRW_exponent=0.9, n_steps=1000, speed=6, s_pos=[0,0])

In [47]:
# aux to store turning angles
aux_ta_CRW_2d_df_6 = np.empty(shape=(0))

# Iterate over trajectory compute turning angles
for index, row in CRW_2d_df_6[1:-1].iterrows():
    ## start - Add your code here
    angle = turning_angle(CRW_2d_df_6.iloc[index-1], row, CRW_2d_df_6.iloc[index+1])
    aux_ta_CRW_2d_df_6 = np.append(aux_ta_CRW_2d_df_6,angle)
    ## end - Add your code here

# aux to store turning angles
aux_ta_CRW_2d_df_9 = np.empty(shape=(0))
for index, row in CRW_2d_df_9[1:-1].iterrows():
    ## start - Add your code here
    angle = turning_angle(CRW_2d_df_9.iloc[index-1], row, CRW_2d_df_9.iloc[index+1])
    aux_ta_CRW_2d_df_9 = np.append(aux_ta_CRW_2d_df_9,angle)
## end - Add your code here

In [48]:
# Define your function to compute Turning Angles for given trajectory

## start - Add your code here
def turning_angle_trajectory(trajectory):
    angles = pd.DataFrame(columns=['angle'])
    for i, row in trajectory[1:-1].iterrows():
        angle = turning_angle(trajectory.iloc[i-1], row, trajectory.iloc[i+1])
        temp_df = pd.DataFrame([{'angle': angle}])
        angles = pd.concat([angles,temp_df], ignore_index=True)
    return angles
## end - Add your code here

In [49]:
# Get Turning Angles calling the function
ta_CRW_2d_df_6 = turning_angle_trajectory(CRW_2d_df_6)

# Get Turning Angles calling the function
ta_CRW_2d_df_9 = turning_angle_trajectory(CRW_2d_df_9)

C:\Users\Rogertrog\AppData\Local\Temp\ipykernel_13780\1179491435.py:9: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

C:\Users\Rogertrog\AppData\Local\Temp\ipykernel_13780\1179491435.py:9: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [50]:
# Check documentation
# https://plotly.com/python/histograms/

# PLot histogram
fig_met_df_3 = go.Figure()

# Histogram turning angle CRW_2d_df_6
## start - Add your code here
fig_met_df_3.add_trace(go.Histogram(
    x=ta_CRW_2d_df_6['angle'],
    marker=dict(
        color='blue'
    ),
    opacity=0.5,
    name='ta_CRW_2d_df_6')
    )
## end - Add your code here


# Histogram turning angle CRW_2d_df_9
## start - Add your code here
fig_met_df_3.add_trace(go.Histogram(
    x=ta_CRW_2d_df_9['angle'],
    marker=dict(
        color='red'
    ),
    opacity=0.5,
    name='ta_CRW_2d_df_9')
)
## end - Add your code here

# Add origin distributions
## start - Add your code here
def wrapped_cauchy_pdf(theta, rho):
    numerator = (1 - rho**2)
    denominator = (2 * np.pi * (1 + rho**2 - 2 * rho * np.cos(theta)))
    return numerator / denominator

theta_range = np.linspace(-np.pi, np.pi, len(ta_CRW_2d_df_6['angle']))
pdf_6 = wrapped_cauchy_pdf(theta_range, 0.6)
pdf_9 = wrapped_cauchy_pdf(theta_range, 0.9)

count_6, bins_6 = np.histogram(ta_CRW_2d_df_6['angle'], bins=30, density=False)
count_9, bins_9 = np.histogram(ta_CRW_2d_df_9['angle'], bins=30, density=False)

pdf_6_scaled = pdf_6 * (max(count_6) / max(pdf_6))
pdf_9_scaled = pdf_9 * (max(count_9) / max(pdf_9))

fig_met_df_3.add_trace(go.Scatter(x=theta_range, y=pdf_6_scaled, mode='lines', name='Cauchy_6', marker_color='blue'))
fig_met_df_3.add_trace(go.Scatter(x=theta_range, y=pdf_9_scaled, mode='lines', name='Cauchy_9', marker_color='red'))
fig_met_df_3.update_layout(title='Turning Angle Distribution',
                  bargap=0.2, barmode='overlay')
## end - Add your code here
fig_met_df_3.show()

## Actividad 4: Step-length Distribution - (Dist. origen vs Dist. observada) (6 pts)

* Implementar función que genere **Lévy Walks** (LW) utilizando pandas.
* Guardar trayectorias en Pandas Data Frame.
* Implementar una función que calcule los valores de **step lenght** de una trayectoria.
* Guardar trayectorias en **pandas** Data Frame.
* Obtener **Step-length** distribution.
* Comparar en gráfica distribución origen vs distribución observada (Histograma).
* Visualizar con plotly.

In [6]:
# Load existing trajectories to test your implementation
# Levy speed = 6
# levy_stable [alpha=1.0, beta=1.0, loc=3.0]
Levy_2d_df_1 = pd.read_csv('files/trajectories/levy_6_1.csv')

# Load existing trajectories to test your implementation
# Levy speed = 6
# levy_stable [alpha=0.7, beta=1.0, loc=3.0]
Levy_2d_df_7 = pd.read_csv('files/trajectories/levy_6_7.csv')

In [7]:
# Define your function to compute Step lengths for given trajectory

## start - Add your code here
def turning_angle_trajectory_list(trajectory):
    angles = []
    for i, row in trajectory[1:-1].iterrows():
        angle = turning_angle(trajectory.iloc[i-1], row, trajectory.iloc[i+1])
        angles.append(angle)
    angles_df = pd.DataFrame(angles, columns=['angles'])
    return angles_df

def step_length(angles):
  step_count = 0
  steps = []
  for angle in angles['angles']:
      step_count += 1
      if(np.abs(angle) != 0):
          steps.append(step_count)
          step_count = 0
  steps_df = pd.DataFrame(steps, columns=['steps'])
  return steps_df

## end - Add your code here

In [8]:
# Get Step lengths calling the function
angles_1 = turning_angle_trajectory_list(Levy_2d_df_1)
sl_Levy_2d_df_1 = step_length(angles_1)

# Get Step lengths calling the function
angles_7 = turning_angle_trajectory_list(Levy_2d_df_7)
sl_Levy_2d_df_7 = step_length(angles_7)

In [9]:
# PLot histogram
fig_met_df_4 = go.Figure()

# Histogram step-length Levy_2d_df_1
## start - Add your code here

## end - Add your code here
fig_met_df_4.add_trace(go.Histogram(x=sl_Levy_2d_df_1['steps'], name='SL_Levy_1', 
    marker_color='blue', opacity=0.5))
# Histogram step-length Levy_2d_df_7
## start - Add your code here
fig_met_df_4.add_trace(go.Histogram(x=sl_Levy_2d_df_7['steps'], name='SL_Levy_7', 
    marker_color='red', opacity=0.5))
## end - Add your code here

# Add origin distributions
## start - Add your code here

## end - Add your code here

fig_met_df_4.show()